In [ ]:
import kserve

kserve.ModelServer(http_port=8100, enable_docs_url=True)

In [9]:
!pip install kserve==0.14.0

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: kserve
    Found existing installation: kserve 0.14.1
    Uninstalling kserve-0.14.1:
      Successfully uninstalled kserve-0.14.1

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [34]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
from opensearchpy import OpenSearch, helpers
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
opensearch_params = {
    "host": "localhost",
    "port": "9200",
    "username": os.environ["OPENSEARCH_USERNAME"],
    "password": os.environ["OPENSEARCH_PASSWORD"],
}

In [ ]:
opensearch =

In [3]:
opensearch_client = OpenSearch(
    hosts=[{"host": opensearch_params["host"], "port": opensearch_params["port"]}],
    http_compress=False,
    http_auth=(opensearch_params["username"], opensearch_params["password"]),
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

In [31]:
INDEX_NAME = "profagro"

In [40]:
# Имя индекса, который нужно удалить
index_name = "profagro_amazone_kserve_baai_bge_m3"


response = opensearch_client.indices.create(index=index_name)

In [43]:
response

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'profagro_amazone_kserve_baai_bge_m3'}

In [55]:
opensearch_client.indices.refresh(index=index_name)

{'_shards': {'total': 3, 'successful': 3, 'failed': 0}}

In [4]:
print(list(opensearch_client.indices.get_alias("*").keys()))


['.opensearch-observability', '.plugins-ml-config', '.opensearch-sap-log-types-config', 'profagro_kserve_baai_bge_m3_amazone', 'profagro_kserve_baai_bge_m3_kverneland', '.kibana_1', '.opendistro_security']


In [32]:
get_opensearch_default_index_settings = lambda x: {
    "settings": {
        "index": {"number_of_shards": 2},
        "analysis": {
            "analyzer": {
                "default": {
                    "tokenizer": "standard",
                    "filter": ["lowercase", "russian_snowball"],
                },
                "default_search": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "russian_stop",
                        "russian_snowball",
                        "graph_synonyms",
                    ],
                },
            },
            "filter": {
                "graph_synonyms": {
                    "type": "index_synonym_graph",
                    "index": ".synonyms",
                    "expand": True,
                    "lenient": False,
                    "username": opensearch_params["username"],
                    "password": opensearch_params["password"],
                },
                "russian_snowball": {"type": "snowball", "language": "Russian"},
                "russian_stop": {"type": "stop", "stopwords": "_russian_"},
                "shingle": {
                    "type": "shingle",
                    "min_shingle_size": 2,
                    "max_shingle_size": 2,
                    "output_unigrams": True,
                },
            },
        },
    },
}

In [53]:
for index_name in opensearch_client.indices.get_alias("*").keys():
    if ".synonyms" in index_name or INDEX_NAME in index_name:
        response = opensearch_client.indices.delete(index=index_name, ignore=[400, 404])
        print(response)

{'acknowledged': True}
{'acknowledged': True}


In [54]:
import requests
import json

result = requests.put(
    f"https://{opensearch_params['host']}:{opensearch_params['port']}/{INDEX_NAME}",
    json=get_opensearch_default_index_settings(INDEX_NAME),
    auth=(opensearch_params["username"], opensearch_params["password"]),
    verify=False,
)

print(result.text)

{"acknowledged":true,"shards_acknowledged":true,"index":"profagro"}


In [5]:
import requests

synonyms = []

result = requests.post(
    f"https://{opensearch_params['host']}:{opensearch_params['port']}/.synonyms/_doc/synonyms",
    json={"synonyms": synonyms},
    auth=(opensearch_params["username"], opensearch_params["password"]),
    verify=False,
)
if result.status_code not in [200, 201]:
    print(f"Synonyms update failed: {result.status_code} - {result.text}")

In [35]:
result = requests.post(
    f"https://{opensearch_params['host']}:{opensearch_params['port']}/_plugins/_refresh_search_analyzers/{INDEX_NAME}",
    auth=(opensearch_params["username"], opensearch_params["password"]),
    verify=False,
)
print(result.text)

{"_shards":{"total":4,"successful":4,"failed":0},"successful_refresh_details":[{"index":"profagro","refreshed_analyzers":["default_search"]}]}


In [36]:
query = {"query": {"match": {"content": "amazone"}}}

response = opensearch_client.search(body=query, index="profagro")

print(f"Total hits: {response['hits']['total']['value']}")
print("\nRetrieved documents:")
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}")
    print(f"Score: {hit['_score']}")
    print(f"Content: {hit['_source']['content']}")
    print("-" * 50)

Total hits: 0

Retrieved documents:
